# Mike Babb
# babbm@uw.edu
# Introduction to Python Part 02

In [ ]:
# we'll need three python libraries to help with this: os, pandas, and numpy
import os # operating system

In [ ]:
import pandas as pd # pd is an alias for pandas
import numpy as np # np is an alias for numpy

# we can use the geopandas library to plot geographic data
# http://geopandas.org/
import geopandas as gpd # gpd is the alias for geopandas

# we'll plot using the matplot lib library
# https://matplotlib.org/api/_as_gen/matplotlib.pyplot.boxplot.html#matplotlib.pyplot.boxplot
import matplotlib.pyplot as plt # plt is the alias for matplotlib.pyplot
%matplotlib inline # control how plotting behaves

## working with data

In [ ]:
# So far we've entered data and values directly into a Jupyter Notbook cell. 
# What if we want to read data into that exist elsewhere?
# Let's read in data pertaining to sex by age groups for Census Designated Places in
# Washington state during the 2013-2017 time period.
# These data come from the ACS and were downloaded from American Fact Finder:
# https://factfinder.census.gov/faces/nav/jsf/pages/index.xhtml

# More information on Census Designated Places
# https://www.census.gov/geo/reference/gtc/gtc_place.html

## specify your netid

In [ ]:
netid = 'babbm'

In [ ]:
# First, we're going to create a string varible with the value of the file directory
# we're working with. Adjust the path accordingly.
file_path = os.path.join('C:/Users', netid, 'documents/intro_to_python-master/intro_to_python/aff_download')

In [ ]:
# let's check the value
file_path

In [ ]:
# specify the name of the file
file_name = 'ACS_17_5YR_S0101_with_ann.csv'

In [ ]:
# join the file path and file name together using a function
file_path_name = os.path.join(file_path, file_name)
file_path_name

In [ ]:
# this isn't necessary for this tutorial, but sometimes it's important to display the current
# values of variables. The fully qualified file path is difficult to read with the mixture
# of forward and backward slashes. python includes many built in functions to
# format things for easy reading and display
os.path.normpath(file_path_name)

### reading data using pandas

In [ ]:
# we know our data are rectangular and parsing that information by hand is going to take
# too much time and effort.
# The pandas library to the rescue!
# https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html

# Specifying the dtype=np.str is telling pandas to read all of our data in as strings
# this is necessary due to how AFF data are delivered to the end user. 
df = pd.read_csv(filepath_or_buffer=file_path_name, sep=',', header=0, dtype=np.str)

In [ ]:
# let's see the first few rows
df.head()

In [ ]:
# how many records?
len(df)

In [ ]:
# There is a lot going on here!
# And it looks like the first line in our data file features the stats software friendly
# names of columns and the second line features descriptive names.
# This information is repeated in the ACS_17_5YR_S0101_metadata.csv file as well.

In [ ]:
# we'll access our data by integer location
# https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.iloc.html
# remember, dataframes are zero-indexed. 
# we'll get lines 1 through the end and remove the zeroeth record.
df = df.iloc[1:]

In [ ]:
df.head()

### selecting and renaming columns

In [ ]:
# 459 columns. that's a lot.
# As we are working with survey data, we have the estimate (EST),
# and the margin of error (MOE) for each value.
# The metadata file, ACS_17_5YR_S0101_metadata.csv, describes each variable.

In [ ]:
# It looks like we have a combination of upper and lower case column names.
# let's convert everything to lower case to make things easier to work with

In [ ]:
# get column names
col_names = df.columns.tolist()

In [ ]:
col_names

In [ ]:
# let's use a loop and during each iteration convert the current column name to lower case
# and append items to a new list
new_col_names = [] # empty list to hold our output
for cn in col_names:
    new_cn = cn.lower()
    new_col_names.append(new_cn)

In [ ]:
new_col_names

In [ ]:
# better.
# another way to make lower case names is through list comprehension.
# https://docs.python.org/3/tutorial/datastructures.html#list-comprehensions
# one of python's more effective and efficient techniques
# while the outcome is equivalent, it's often more tidy (and frequently faster)
# to use list comprehension
new_col_names = [cn.lower() for cn in col_names]

In [ ]:
df.columns = new_col_names

In [ ]:
df.head()

In [ ]:
# As we're calculating the all-ages sex ratio, we probably do not need all 459 columns.
# Let's consult our metadata document, ACS_17_5YR_S0101_metadata.csv, to see how 
# we can trim down our list of variables

In [ ]:
# let's keep the following:
# geo.id - place identifier
# geo.id2 - place identifier
# geo.display-label - expanded description of the place
# hc01_est_vc01 - total population
# hc03_est_vc01 - total males
# hc05_est_vc01 - total females
# place identification variables, total population, total males, and total females
col_names = ['geo.id', 'geo.id2', 'geo.display-label', 'hc01_est_vc01','hc03_est_vc01','hc05_est_vc01']

In [ ]:
# let's create a copy of our data and work with only a subset of varables
# we're selecting using a list of column names
working_df = df[col_names]

In [ ]:
working_df.head()

In [ ]:
# let's rename some of our columns to help with producing the statistics of interest
# we'll use a python dictionary to store how we will rename things
# https://docs.python.org/3/tutorial/datastructures.html#dictionaries
# other programming languages call these objects 'hash tables' or 'associative arrays'
# https://en.wikipedia.org/wiki/Associative_array

In [ ]:
rename_dictionary = {'hc01_est_vc01':'total_pop', 'hc03_est_vc01':'males', 'hc05_est_vc01':'females'}

In [ ]:
# specify the input value and the dictionary will tell you the associated value/object
rename_dictionary['hc01_est_vc01']

In [ ]:
# rename
working_df = working_df.rename(columns=rename_dictionary)

In [ ]:
working_df.head()

In [ ]:
# much better. 

### computing basic statistics

In [ ]:
# how many people in Washington are there in total?
working_df['total_pop'].sum()

In [ ]:
# we just concatenated all of the strings together!
# we need to change the data type first
working_df['total_pop'] = working_df['total_pop'].astype(np.int32) # specify a 32-bit integer

In [ ]:
working_df['total_pop'].sum()

In [ ]:
# compare with more current estimates
# https://www.census.gov/quickfacts/wa
# 7,535,591 as of July 1, 2018
# part of the difference is due to the geography we are using: census designated places.
# this mostly excludes rural populations

In [ ]:
# what are some summary statistics at the place level?
working_df['total_pop'].describe()

In [ ]:
# how many males in total?
working_df['males'] = working_df['males'].astype(np.int32)
working_df['males'].sum()

In [ ]:
# how many females?
working_df['females'] = working_df['females'].astype(np.int32)
working_df['females'].sum()

In [ ]:
# more females than males - not surprising given that this is data for all ages
# females tend to live longer than males

### computing the all ages sex ratio

In [ ]:
# compute the sex ratio: the number of males per 100 females
working_df['sex_ratio'] = (working_df['males'] / working_df['females']) * 100

In [ ]:
working_df['sex_ratio'].describe()

In [ ]:
# the inf and NAN values usually indicate a division by zero error.
# Are there places with zero females?

In [ ]:
working_df.head()

### identifying outliers

In [ ]:
# back to our descriptions
working_df['males'].describe()

In [ ]:
working_df['females'].describe()

In [ ]:
working_df['total_pop'].describe()

In [ ]:
# are there census designated places with zero people?
working_df[working_df['total_pop']==0].head()

In [ ]:
# are there the same as the towns with zero males?
working_df[working_df['males']==0].head()

In [ ]:
# zero females?
working_df[working_df['females']==0].head()

In [ ]:
# for these areas with zero people:
# should we remove? 
# should we flag?
# let's remove these data for now.

In [ ]:
# let's remove the places with zero population and zero females
working_df = working_df[working_df['total_pop']>0]
working_df = working_df[working_df['females']>0]

### applying a user a defined function

In [ ]:
# but we can't compute the sex ratio as we did above.
# up until now, we've been operating on columns (vectors) of our data.
# to compute the ratios, we'll need to write a function that examines each row in our pandas dataframe

# we're now going to apply a user defined function to reach row in our dataframe
# https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.apply.html

In [ ]:
def sex_ratio(row):
    males = row['males'] # the current row's 'males' value
    females = row['females'] # current row's 'females' value
    # use an if statement to account for places with 0 females surveyed.
    if females == 0:
        # we'll add 1 to the count of zero, in this case.
        outcome = (males / (females + 1)) * 100
    else:
        outcome = (males / females) * 100
    
    return outcome

In [ ]:
working_df['sex_ratio'] = working_df.apply(func=sex_ratio, axis=1)

In [ ]:
working_df['sex_ratio'].describe()

In [ ]:
# what is going with the max value?
working_df.sort_values(by='sex_ratio', ascending=False).head()

In [ ]:
# let's say we want to look at the sex ratio by quintiles of population size
# maybe there is something systematic going on in smaller areas?
# https://pandas.pydata.org/pandas-docs/version/0.23.4/generated/pandas.qcut.html
working_df['pop_quintile'] = pd.qcut(x=working_df['total_pop'], q=5, labels=False)

In [ ]:
working_df.head()

In [ ]:
# let's see our labels for our quintiles
working_df['pop_quintile'].unique()

In [ ]:
# python is zero-indexed. Let's enumerate each quintile and print the summary statistics
for i in range(0, 5):
    # apply string formatting to produce a meaningful descriptor
    curr_quintile = (i + 1) * .2
    # use the format() function to display information in a more intuitive format
    # https://docs.python.org/3.7/library/string.html#format-string-syntax
    curr_quintile = '{:.0%}'.format(curr_quintile)
    
    # let's look at the current quintile
    curr_df = working_df[working_df['pop_quintile']==i]
        
    print('****TOTAL POPULATION****')
    print('Current quintile:', curr_quintile)
    print(curr_df['sex_ratio'].describe())   
    print()
    

In [ ]:
# it does look like there is something odd about the smaller places based
# on descriptive statistics
# but let's visualize the descriptive statistics 

### plotting data

In [ ]:
# let's plot this using the matplot lib library
# https://matplotlib.org/api/_as_gen/matplotlib.pyplot.boxplot.html#matplotlib.pyplot.boxplot
import matplotlib.pyplot as plt
%matplotlib inline 

In [ ]:
boxplot = working_df.boxplot(column=['sex_ratio'], by=['pop_quintile'])

In [ ]:
# that looks great.
# But what if we wanted to compute deciles?

In [ ]:
working_df['pop_decile'] = pd.qcut(x=working_df['total_pop'], q=10, labels=False)

In [ ]:
# another box plot
boxplot = working_df.boxplot(column=['sex_ratio'], by=['pop_decile'])

# but this time format and label the axes and title
boxplot.set_xlabel('Population Deciles') # format the x axis label
boxplot.set_ylabel('Number of Males per 100 Females') # format the y axis label
boxplot.set_title('Avg. Estimated Sex Ratio in Washington State Places, 2013 - 2017') # add a title
boxplot.figure.suptitle('') # set the subtitle to an empty string - to help with the aesthetic

# plotting geographic data

In [ ]:
# we can use the geopandas library to plot geographic data
# http://geopandas.org/
import geopandas as gpd

In [ ]:
# First, we're going to create a string varible with the value of the directory we're working with
s_file_path = os.path.join('C:/Users', netid, 'documents/intro_to_python-master/intro_to_python/tl_2018_53_place')

In [ ]:
s_file_name = 'tl_2018_53_place.shp'
s_fpn = os.path.join(s_file_path, s_file_name)

In [ ]:
gdf = gpd.read_file(filename=s_fpn)

In [ ]:
gdf.plot()

In [ ]:
# that's really hard to see, even if we were to zoom in. 
# let's format our data so that we can join the sex ratio data
# to the geographic data and then export it to use in GIS

In [ ]:
# what are our column names in the shapefile?
gdf.head(n=100)

In [ ]:
# The geodataframe features a field named GEOID.
# this matches 'geoid' in our working dataframe, but the field names are of different case.
# Let's double check.

In [ ]:
working_df.head()

In [ ]:
# no! it's geo.id2
# let's clean up the dataframe with the sex ratios

In [ ]:
# drop the geo.id field - it's superflous at this point
working_df = working_df.drop('geo.id', 1)

In [ ]:
# check
working_df.head()

In [ ]:
# it also looks like 'geo.display-label' features the same information as 'NAME'
# and 'NAMELSAD' in the geodataframe. Let's keep it, but rename as well.

In [ ]:
# rename
rename_columns = {'geo.id2':'geoid', 'geo.display-label':'areaname'}
working_df = working_df.rename(columns=rename_columns)

In [ ]:
working_df.head()

In [ ]:
# better...
# because we are ultimately saving this data to a shapefile, we need to do a little more prep
# shapefiles are a very inflexible data storage mechanism
# https://en.wikipedia.org/wiki/Shapefile
# field names must be 10 characters or less
# field names CANNOT start with a number
# field names CANNOT contain the following characters: ,./<>?;':"[]\{}|`~!@#$%^&*()-=+"'
# field names CAN contain an underscore _
# field names prefer to use UPPER-CASE LETTERS

In [ ]:
# create uppercase columns
col_names = working_df.columns.tolist()

In [ ]:
# use list comprehension to convert our field names to uppercase
col_names = [x.upper() for x in col_names]

In [ ]:
working_df.columns = col_names

In [ ]:
working_df.head()

In [ ]:
# hmmm, I think that some of those field names are longer than 10 characters
# better rename just to be on the safe side
rename_columns = {'POP_QUINTILE':'P_QUINTILE', 'POP_DECILE':'P_DECILE'}
working_df = working_df.rename(columns=rename_columns)

In [ ]:
working_df.head()

In [ ]:
# join the sex ratio data to the geographic data
gdf = pd.merge(left=gdf, right=working_df, how='left')

In [ ]:
gdf.head()

In [ ]:
# specify the output name
output_s_file_name = 'wa_places_sex_ratio.shp'
output_s_fpn = os.path.join(s_file_path, output_s_file_name)

In [ ]:
# write the geodataframe to disk
gdf.to_file(filename=output_s_fpn, driver='ESRI Shapefile')

### exporting data

In [ ]:
# we can also save the data to a csv and an excel file
# first, to the csv
file_name = 'wa_places_sex_ratios_2013_2017.csv'
file_path_name = os.path.join(file_path, file_name)

In [ ]:
# https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_csv.html
working_df.to_csv(path_or_buf=file_path_name, sep='\t', index=False)

In [ ]:
# now, to excel
file_name = 'wa_places_sex_ratios_2013_2017.xlsx'
file_path_name = os.path.join(file_path, file_name)

In [ ]:
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_excel.html
working_df.to_excel(excel_writer=file_path_name, sheet_name='wa_places_sex_ratio_2013_2017', index=False)

# let's use qGIS to take a look at the data!

In [ ]:
# https://www.qgis.org/en/site/